In [124]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime, timedelta
#para request
import requests
#parametros
import parametros
import time

## Leyendo datos del indice ohmyfi-detalleconexiones

In [125]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

### Definiendo fechas para rangos de consultas

In [126]:
now = datetime.now()
new_date = now - timedelta(days=6)
ahora = str(now.strftime("%Y/%m/%d %H:%M"))
fecha_6 = str(new_date.strftime("%Y/%m/%d %H:%M"))

## tomando fecha mas reciente del indice

In [127]:
total_docs = 0
try:
    response = es.search(
        index=parametros.ohmyfi_d_c_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "fechahora", "format": "yyyy/MM/dd HH:mm"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max = response["aggregations"]["max_date"]['value_as_string']
    h_inicio = response["aggregations"]["max_date"]['value_as_string']
    h_fin = (datetime.strptime(h_inicio, '%Y/%m/%d %H:%M')+timedelta(hours=1)).strftime("%Y/%m/%d %H:%M")
except:
    fecha_max = fecha_6
    h_inicio = fecha_6
    h_fin = str((new_date+timedelta(hours=1)).strftime("%Y/%m/%d %H:%M"))

print("Fecha maxima en indice:", fecha_max,"| Fecha inicio:", h_inicio, " | Fecha fin:", h_fin)

Fecha maxima en indice: 2021/10/20 10:00 | Fecha inicio: 2021/10/20 10:00  | Fecha fin: 2021/10/20 11:00


In [128]:
h_inicio, h_fin, ahora

('2021/10/20 10:00', '2021/10/20 11:00', '2021/10/20 10:02')

## Leyendo la APi detalleconexiones

In [129]:
url2 = parametros.url_ohmyfi +'?/detalleconexiones/=&apiKey=' + parametros.ohmyfi_api_key 

In [130]:
datos_api = pd.DataFrame(columns=['mac_usuario','nombre_completo','apellidos','correo','documento','tipodoc'
                                      ,'celular','genero','fecha_nacimiento','dispositivo','sistema_operativo'
                                      ,'fecha','fechahora','lugar','tipo','id_lugar','lugar_cod'])
iterar = 0
while (h_inicio < ahora) & (iterar < 27):
    iterar = iterar + 1
    if h_fin[11:16] == '00:59':
        h_inicio = h_fin[0:11] + '00:00'
    elif h_inicio[8:10] != h_fin[8:10]:
        h_inicio = h_inicio[0:11] + '23:00'
        h_fin = h_inicio[0:11] + '23:59'
    url = url2 + '&inicio=' + str(h_inicio) + '&final=' + str(h_fin) 
    r = requests.get(url, timeout=15)
    if r.status_code == 200:
        res = json.loads(r.text)
        print(h_inicio)
        datos_api = datos_api.append(res, ignore_index=True)
        #datos_api.drop_duplicates(inplace=True)
    else:
        print("Error de request:",r.status_code)
        if r.status_code != 204:
            print("Error:",r.status_code,"URL:",url)
        #exit()
    h_inicio = h_fin
    h_fin = (datetime.strptime(h_inicio, '%Y/%m/%d %H:%M') + timedelta(hours=1)).strftime("%Y/%m/%d %H:%M")
    time.sleep(3)

2021/10/20 10:00


In [131]:
url

'https://www.ohmyfi.com/ApiOMF?/detalleconexiones/=&apiKey=okMOpLAkiYpafQKXhXirwUys&inicio=2021/10/20 10:00&final=2021/10/20 11:00'

In [132]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,0A-76-09-0B-28-BC,Ana,Martinez,maryurisbegambre@gmail.com,1007415810,CC,3205845369,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:00.029015-05,DIVIDIVI_SAHAGUN,True,9683,74108-VZVF061,10:00:00
1,7C-89-56-47-39-1A,Banesa,Albarracin,carolromero553@gmail.com,1324768934,TI,3219937377,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:00.442721-05,CORREG SAN JUANCITO_TEORAMA,True,7816,43446-VZVF988,10:00:00
2,CC-61-E5-D6-9D-F1,Norma,None,egidiovillanueva@gmail.com,1100083062,CC,3234547009,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:00.752274-05,EL PITAL_SAN MARCOS,True,8374,47536-ZIFV499,10:00:00
3,4C-63-71-52-CE-C2,Alexandra,Pacheco,pachecopachecoalexandra@gmail.com,1007912950,CC,3214680143,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:01.027693-05,VDA EL HIGUERON_ABREGO,True,9095,41984-ZIFV249\t,10:00:00
4,5C-D0-6E-F2-16-92,Johan,None,None,11092948055,TI,None,m,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:01.123962-05,SAN BERNARDO_FRESNO,True,6030,48533-ZZZY307,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,52-97-6E-D1-74-A4,Lus,Garcia,None,1007512095,Otra,None,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06.649769-05,VEREDA SAN BERNARDO_CIMITARRA,True,10059,45209-KPWM923,10:00:00
838,36-86-F7-28-68-68,DALILA,MARTINEZ,mdalila020@gmail.com,1003233071,CC,3135250991,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06.84752-05,GUATAPURÍ_VALLEDUPAR,True,7601,30056-ZGYO888,10:00:00
839,F4-71-90-93-DD-2C,valeria,marquez serpa,None,1072249384,TI,3206290471,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:03:06.914084-05,SAN JUAN DE LA CRUZ_TUCHIN,True,9398,32278-VZVF900,10:00:00
840,7A-83-CA-FA-24-7D,Sandy,Beleño,belenoleguiasandymaria@gmail.com,1100012130,CC,573208411714,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:03:07.114172-05,VDA LA CEJA_GUARANDA,True,8062,47178-VZVF171,10:00:00


In [133]:
datos_api.to_excel("detalles_crudo_14_10_21.xlsx")

### Descartando datos de la API que ya están en el indice

Si el dataframe datos_api no tiene datos, cae en el except.

In [134]:
h_inicio, h_fin, ahora
#datos_api2 = datos_api

('2021/10/20 11:00', '2021/10/20 12:00', '2021/10/20 10:02')

In [135]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,0A-76-09-0B-28-BC,Ana,Martinez,maryurisbegambre@gmail.com,1007415810,CC,3205845369,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:00.029015-05,DIVIDIVI_SAHAGUN,True,9683,74108-VZVF061,10:00:00
1,7C-89-56-47-39-1A,Banesa,Albarracin,carolromero553@gmail.com,1324768934,TI,3219937377,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:00.442721-05,CORREG SAN JUANCITO_TEORAMA,True,7816,43446-VZVF988,10:00:00
2,CC-61-E5-D6-9D-F1,Norma,None,egidiovillanueva@gmail.com,1100083062,CC,3234547009,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:00.752274-05,EL PITAL_SAN MARCOS,True,8374,47536-ZIFV499,10:00:00
3,4C-63-71-52-CE-C2,Alexandra,Pacheco,pachecopachecoalexandra@gmail.com,1007912950,CC,3214680143,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:01.027693-05,VDA EL HIGUERON_ABREGO,True,9095,41984-ZIFV249\t,10:00:00
4,5C-D0-6E-F2-16-92,Johan,None,None,11092948055,TI,None,m,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:01.123962-05,SAN BERNARDO_FRESNO,True,6030,48533-ZZZY307,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,52-97-6E-D1-74-A4,Lus,Garcia,None,1007512095,Otra,None,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06.649769-05,VEREDA SAN BERNARDO_CIMITARRA,True,10059,45209-KPWM923,10:00:00
838,36-86-F7-28-68-68,DALILA,MARTINEZ,mdalila020@gmail.com,1003233071,CC,3135250991,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06.84752-05,GUATAPURÍ_VALLEDUPAR,True,7601,30056-ZGYO888,10:00:00
839,F4-71-90-93-DD-2C,valeria,marquez serpa,None,1072249384,TI,3206290471,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:03:06.914084-05,SAN JUAN DE LA CRUZ_TUCHIN,True,9398,32278-VZVF900,10:00:00
840,7A-83-CA-FA-24-7D,Sandy,Beleño,belenoleguiasandymaria@gmail.com,1100012130,CC,573208411714,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:03:07.114172-05,VDA LA CEJA_GUARANDA,True,8062,47178-VZVF171,10:00:00


In [136]:
datos_api.drop_duplicates(inplace=True)
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,0A-76-09-0B-28-BC,Ana,Martinez,maryurisbegambre@gmail.com,1007415810,CC,3205845369,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:00.029015-05,DIVIDIVI_SAHAGUN,True,9683,74108-VZVF061,10:00:00
1,7C-89-56-47-39-1A,Banesa,Albarracin,carolromero553@gmail.com,1324768934,TI,3219937377,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:00.442721-05,CORREG SAN JUANCITO_TEORAMA,True,7816,43446-VZVF988,10:00:00
2,CC-61-E5-D6-9D-F1,Norma,None,egidiovillanueva@gmail.com,1100083062,CC,3234547009,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:00.752274-05,EL PITAL_SAN MARCOS,True,8374,47536-ZIFV499,10:00:00
3,4C-63-71-52-CE-C2,Alexandra,Pacheco,pachecopachecoalexandra@gmail.com,1007912950,CC,3214680143,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:01.027693-05,VDA EL HIGUERON_ABREGO,True,9095,41984-ZIFV249\t,10:00:00
4,5C-D0-6E-F2-16-92,Johan,None,None,11092948055,TI,None,m,None,Smartphone,Android,2021-10-20,2021-10-20 10:00:01.123962-05,SAN BERNARDO_FRESNO,True,6030,48533-ZZZY307,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,52-97-6E-D1-74-A4,Lus,Garcia,None,1007512095,Otra,None,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06.649769-05,VEREDA SAN BERNARDO_CIMITARRA,True,10059,45209-KPWM923,10:00:00
838,36-86-F7-28-68-68,DALILA,MARTINEZ,mdalila020@gmail.com,1003233071,CC,3135250991,f,None,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06.84752-05,GUATAPURÍ_VALLEDUPAR,True,7601,30056-ZGYO888,10:00:00
839,F4-71-90-93-DD-2C,valeria,marquez serpa,None,1072249384,TI,3206290471,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:03:06.914084-05,SAN JUAN DE LA CRUZ_TUCHIN,True,9398,32278-VZVF900,10:00:00
840,7A-83-CA-FA-24-7D,Sandy,Beleño,belenoleguiasandymaria@gmail.com,1100012130,CC,573208411714,f,None,Smartphone,Android,2021-10-20,2021-10-20 10:03:07.114172-05,VDA LA CEJA_GUARANDA,True,8062,47178-VZVF171,10:00:00


In [137]:
try:
    datos_api = datos_api.drop_duplicates()
    datos_api.fillna('', inplace=True)
    datos_api['fechahora'] = datos_api["fechahora"].str.split(".", n = 1, expand = True)[0]
    datos_api = datos_api.drop(datos_api[(datos_api['fechahora']<= fecha_max.replace("/","-"))].index)
    datos_api['mac_usuario'] = datos_api['mac_usuario'].str.replace('-',':')
except:
    pass

In [138]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora
0,0A:76:09:0B:28:BC,Ana,Martinez,maryurisbegambre@gmail.com,1007415810,CC,3205845369,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:00,DIVIDIVI_SAHAGUN,True,9683,74108-VZVF061,10:00:00
1,7C:89:56:47:39:1A,Banesa,Albarracin,carolromero553@gmail.com,1324768934,TI,3219937377,f,,Smartphone,Android,2021-10-20,2021-10-20 10:00:00,CORREG SAN JUANCITO_TEORAMA,True,7816,43446-VZVF988,10:00:00
2,CC:61:E5:D6:9D:F1,Norma,,egidiovillanueva@gmail.com,1100083062,CC,3234547009,f,,Smartphone,Android,2021-10-20,2021-10-20 10:00:00,EL PITAL_SAN MARCOS,True,8374,47536-ZIFV499,10:00:00
3,4C:63:71:52:CE:C2,Alexandra,Pacheco,pachecopachecoalexandra@gmail.com,1007912950,CC,3214680143,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:01,VDA EL HIGUERON_ABREGO,True,9095,41984-ZIFV249\t,10:00:00
4,5C:D0:6E:F2:16:92,Johan,,,11092948055,TI,,m,,Smartphone,Android,2021-10-20,2021-10-20 10:00:01,SAN BERNARDO_FRESNO,True,6030,48533-ZZZY307,10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,52:97:6E:D1:74:A4,Lus,Garcia,,1007512095,Otra,,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06,VEREDA SAN BERNARDO_CIMITARRA,True,10059,45209-KPWM923,10:00:00
838,36:86:F7:28:68:68,DALILA,MARTINEZ,mdalila020@gmail.com,1003233071,CC,3135250991,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06,GUATAPURÍ_VALLEDUPAR,True,7601,30056-ZGYO888,10:00:00
839,F4:71:90:93:DD:2C,valeria,marquez serpa,,1072249384,TI,3206290471,f,,Smartphone,Android,2021-10-20,2021-10-20 10:03:06,SAN JUAN DE LA CRUZ_TUCHIN,True,9398,32278-VZVF900,10:00:00
840,7A:83:CA:FA:24:7D,Sandy,Beleño,belenoleguiasandymaria@gmail.com,1100012130,CC,573208411714,f,,Smartphone,Android,2021-10-20,2021-10-20 10:03:07,VDA LA CEJA_GUARANDA,True,8062,47178-VZVF171,10:00:00


### fecha_control se usará para la agrupación cada 10 minutos

In [139]:
datos_api['fecha_control'] = datos_api["fechahora"].str[0:-4] + '0:00'

In [140]:
datos_api

,mac_usuario,nombre_completo,apellidos,correo,documento,tipodoc,celular,genero,fecha_nacimiento,dispositivo,sistema_operativo,fecha,fechahora,lugar,tipo,id_lugar,lugar_cod,hora,fecha_control
0,0A:76:09:0B:28:BC,Ana,Martinez,maryurisbegambre@gmail.com,1007415810,CC,3205845369,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:00,DIVIDIVI_SAHAGUN,True,9683,74108-VZVF061,10:00:00,2021-10-20 10:00:00
1,7C:89:56:47:39:1A,Banesa,Albarracin,carolromero553@gmail.com,1324768934,TI,3219937377,f,,Smartphone,Android,2021-10-20,2021-10-20 10:00:00,CORREG SAN JUANCITO_TEORAMA,True,7816,43446-VZVF988,10:00:00,2021-10-20 10:00:00
2,CC:61:E5:D6:9D:F1,Norma,,egidiovillanueva@gmail.com,1100083062,CC,3234547009,f,,Smartphone,Android,2021-10-20,2021-10-20 10:00:00,EL PITAL_SAN MARCOS,True,8374,47536-ZIFV499,10:00:00,2021-10-20 10:00:00
3,4C:63:71:52:CE:C2,Alexandra,Pacheco,pachecopachecoalexandra@gmail.com,1007912950,CC,3214680143,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:00:01,VDA EL HIGUERON_ABREGO,True,9095,41984-ZIFV249\t,10:00:00,2021-10-20 10:00:00
4,5C:D0:6E:F2:16:92,Johan,,,11092948055,TI,,m,,Smartphone,Android,2021-10-20,2021-10-20 10:00:01,SAN BERNARDO_FRESNO,True,6030,48533-ZZZY307,10:00:00,2021-10-20 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,52:97:6E:D1:74:A4,Lus,Garcia,,1007512095,Otra,,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06,VEREDA SAN BERNARDO_CIMITARRA,True,10059,45209-KPWM923,10:00:00,2021-10-20 10:00:00
838,36:86:F7:28:68:68,DALILA,MARTINEZ,mdalila020@gmail.com,1003233071,CC,3135250991,f,,Smartphone,AndroidOS,2021-10-20,2021-10-20 10:03:06,GUATAPURÍ_VALLEDUPAR,True,7601,30056-ZGYO888,10:00:00,2021-10-20 10:00:00
839,F4:71:90:93:DD:2C,valeria,marquez serpa,,1072249384,TI,3206290471,f,,Smartphone,Android,2021-10-20,2021-10-20 10:03:06,SAN JUAN DE LA CRUZ_TUCHIN,True,9398,32278-VZVF900,10:00:00,2021-10-20 10:00:00
840,7A:83:CA:FA:24:7D,Sandy,Beleño,belenoleguiasandymaria@gmail.com,1100012130,CC,573208411714,f,,Smartphone,Android,2021-10-20,2021-10-20 10:03:07,VDA LA CEJA_GUARANDA,True,8062,47178-VZVF171,10:00:00,2021-10-20 10:00:00


### Insertando datos en el indice
* Cambio quitar fecha (Solicitado por Napoleon [GM])


In [141]:
indice = 'ohmyfi-detalles_pruingesta20_10_21_2'

In [142]:
use_these_keys = ['mac_usuario','nombre_completo','apellidos','correo','documento','tipodoc','celular'
                  ,'genero','fecha_nacimiento','dispositivo','sistema_operativo','fecha','fechahora','fecha_control'
                  ,'lugar','tipo','id_lugar','lugar_cod','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("Fecha: ",now,"- Documentos insertados: ",salida[0])

Fecha:  2021-10-20 10:02:59.892208 - Documentos insertados:  842
